# Remediator

> The main Remediator agent that orchestrates the resolution of IRIS instance problems using IrisDb and Customer agents.

This module implements the Remediator using the OpenAI SDK Agents framework.

In [ ]:
#| default_exp remediator

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, field
from enum import Enum
import json
from openai import OpenAI

## Data Structures

Define the structured formats for communication between agents.

In [ ]:
#| export
class ProblemType(Enum):
    """Types of problems that can occur in IRIS instance."""
    LICENSE = "license"
    CONFIGURATION = "configuration"
    OS_RESOURCE = "os_resource"
    UNKNOWN = "unknown"

In [ ]:
#| export
@dataclass
class Problem:
    """Structured representation of a problem detected in IRIS instance."""
    problem_type: str
    description: str
    severity: str  # 'critical', 'high', 'medium', 'low'
    log_entry: Optional[str] = None
    affected_component: Optional[str] = None
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'problem_type': self.problem_type,
            'description': self.description,
            'severity': self.severity,
            'log_entry': self.log_entry,
            'affected_component': self.affected_component
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Problem':
        return cls(**data)

In [ ]:
#| export
@dataclass
class Resolution:
    """Possible resolution for a problem."""
    action: str
    description: str
    requires_restart: bool = False
    parameters: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'action': self.action,
            'description': self.description,
            'requires_restart': self.requires_restart,
            'parameters': self.parameters
        }
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Resolution':
        return cls(**data)

## Remediator Agent

The main Remediator agent implemented using the OpenAI SDK Agents framework.

In [ ]:
#| export
from openai.agents import Agent

class RemediatorAgent:
    """Main Remediator agent using OpenAI SDK Agents framework."""
    
    def __init__(self, api_key: str, model: str = "gpt-4o"):
        """Initialize the Remediator Agent.
        
        Args:
            api_key: OpenAI API key
            model: Model to use for the agent
        """
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.history: List[Dict[str, Any]] = []
        
        self.agent = Agent(
            name="Remediator",
            model=self.model,
            instructions="Resolve problems with InterSystems IRIS instance."
        )
    
    def log_step(self, step: str, data: Any) -> None:
        """Log a workflow step."""
        self.history.append({'step': step, 'data': data})
    
    async def remediate(self, prompt: str) -> str:
        """Main remediation workflow."""
        self.log_step('remediate_start', {'prompt': prompt})
                
        try:
            response = await self.agent.run(prompt)
            report = str(response)
            self.log_step('remediate_complete', {'report': report})
            return report
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.log_step('remediate_error', {'error': error_msg})
            return error_msg

## Usage Example

In [ ]:
# Example: Create a Problem instance
problem = Problem(
    problem_type="license",
    description="License has expired",
    severity="critical"
)
print(f"Problem: {problem.description}")

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()